# MCL Collaborative Blueprint Demo

This notebook demonstrates the core components of the MCL 2026 Collaborative Blueprint app:
- Data models for attendee responses
- Plotly visualizations (bar chart, scatter plot, heatmap)
- AI synthesis prompt construction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crowbarmassage/mcl-collaborative-blueprint/blob/main/notebooks/demo.ipynb)

## Step 1: Setup

Clone the repo and install dependencies.

In [ ]:
!git clone https://github.com/crowbarmassage/mcl-collaborative-blueprint.git
%cd mcl-collaborative-blueprint
!pip install -e . -q
print("Setup complete!")

## Step 2: Configure API Key

Load the OpenAI API key from Colab Secrets (or enter manually).

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("API key loaded from Colab Secrets")
except Exception:
    from getpass import getpass
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")

## Step 3: Explore the Data Models

See how attendee responses are structured and converted to spreadsheet rows.

In [ ]:
from mcl_blueprint.config import PRIORITY_CATEGORIES, ARCHETYPES, THREAT_OPTIONS
from mcl_blueprint.models import AttendeeResponse, AggregatedData

print("Priority Categories:", PRIORITY_CATEGORIES)
print("\nThreat Options:", THREAT_OPTIONS)
print("\nArchetypes:")
for name, desc in ARCHETYPES.items():
    print(f"  {name}: {desc}")

In [ ]:
# Create a sample attendee response
response = AttendeeResponse(
    session_id="demo-001",
    q1_budgets={
        "Mental Health": 30, "Prayer Space": 20, "Halal Food": 15,
        "Chaplaincy": 10, "Security/Safety": 10, "Admin Access": 10, "Legal Defense": 5,
    },
    q1_reasoning="Mental health is the foundation for all other advocacy.",
    q2_threat="Budget Cuts",
    q2_likelihood=7,
    q2_impact=9,
    q2_trigger="State legislature funding review",
    q3_archetype="The Ostrich",
    q3_followup="Creates a vacuum where students have no guidance.",
)

print("Response row for Google Sheets:")
print(response.to_row())

## Step 4: Visualizations

Generate the three dashboard charts from sample aggregated data.

In [ ]:
from mcl_blueprint.visualizations import (
    build_priority_bar_chart,
    build_threat_scatter,
    build_archetype_grid,
)

# Create sample aggregated data
agg = AggregatedData(
    total_responses=15,
    avg_budgets={
        "Mental Health": 25.0, "Prayer Space": 20.0, "Halal Food": 18.0,
        "Chaplaincy": 12.0, "Security/Safety": 10.0, "Admin Access": 8.0, "Legal Defense": 7.0,
    },
    threats=[
        ("Budget Cuts", 7.0, 9.0, "State funding review"),
        ("Doxxing", 6.0, 8.0, "Social media campaign"),
        ("Surveillance", 5.0, 7.0, "New campus policy"),
        ("Apathy", 3.0, 4.0, "Student burnout"),
        ("Protest Bans", 8.0, 6.0, "New campus regulations"),
    ],
    archetype_counts={"The Ostrich": 8, "The Fortress": 4, "The Lab": 2, "The Watchtower": 1},
    top_priority="Mental Health",
    top_threat="Budget Cuts",
    dominant_archetype="The Ostrich",
)

In [ ]:
# Q1: Priority Budget Bar Chart
fig1 = build_priority_bar_chart(agg)
fig1.show()

In [ ]:
# Q2: Threat Matrix Scatter Plot
fig2 = build_threat_scatter(agg)
fig2.show()

In [ ]:
# Q3: AI Alignment Archetype Grid
fig3 = build_archetype_grid(agg)
fig3.show()

## Step 5: AI Mirror — Prompt Construction

See the prompt that gets sent to GPT-4o for strategic synthesis.

In [ ]:
from mcl_blueprint.ai_mirror import build_synthesis_prompt

prompt = build_synthesis_prompt(agg)
print("=== Synthesis Prompt ===")
print(prompt)

In [ ]:
# Generate the actual AI tactic (requires OPENAI_API_KEY)
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a concise strategic advisor. Respond in exactly 3 sentences."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=300,
    temperature=0.8,
)

tactic = response.choices[0].message.content
print("=== AI Generated Tactic ===")
print(tactic)

## About

**MCL Collaborative Blueprint** is a real-time conference tool built with Streamlit, Plotly, Google Sheets, and OpenAI.

- **Attendees** answer 3 questions on their phones
- **Dashboard** visualizes aggregate data live on a projector
- **AI Mirror** generates strategic insights from the room's collective intelligence

GitHub: [crowbarmassage/mcl-collaborative-blueprint](https://github.com/crowbarmassage/mcl-collaborative-blueprint)